# Homework 1 (include all parts from challenge 1+optional, 2 and homework)
Authors:
- Nazarii Drushchak
- Igor Babin
- Uliana Zbezhkhovska

In [12]:
!pip install findspark

In [1]:
import findspark
findspark.init()

In [40]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import MinHashLSH
from pyspark.sql.functions import col, avg, when
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.feature import Word2Vec
from sklearn.neighbors import NearestNeighbors

In [29]:
sc = pyspark.SparkContext('local[*]')
spark = SparkSession(sc)
spark

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_323064/950410528.py:1 

## Challenge I

In [30]:
!wget http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/visualisations/listings.csv

--2023-10-20 11:21:44--  http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/visualisations/listings.csv
Resolving data.insideairbnb.com (data.insideairbnb.com)... 52.217.198.45, 54.231.165.53, 54.231.167.29, ...
Connecting to data.insideairbnb.com (data.insideairbnb.com)|52.217.198.45|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1698431 (1.6M) [application/csv]
Saving to: ‘listings.csv.2’

listings.csv.2      100%[===================>]   1.62M  1.62MB/s    in 1.0s    

2023-10-20 11:21:45 (1.62 MB/s) - ‘listings.csv.2’ saved [1698431/1698431]



In [32]:
df = spark.read.csv("listings.csv", header=True, multiLine=True)
df.show(5)

+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+
|    id|                name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|
+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+
|761411|Condo in Amsterda...|4013546|   Xsjong|               NULL|   Noord-Oost|52.40164|  4.95106|   Private room|   61|             3|              303| 2023-08-19|  

Tokenize (remove punctuation and split by word), you can do it in pure python or using ml-lib tokenizer

In [36]:
tokenizer = Tokenizer(inputCol="name", outputCol="words")
wordData = tokenizer.transform(df)
wordData.show(5)

+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+
|    id|                name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|               words|
+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+
|761411|Condo in Amsterda...|4013546|   Xsjong|               NULL|   Noord-Oost|52.40164|  4.95106|   Pri

Remove stopwords using ML-LIB stopwordsremover, and store in a new column called “CleanTokens”

In [37]:
remover = StopWordsRemover(inputCol="words", outputCol="CleanTokens")
cleanData = remover.transform(wordData)
cleanData.show(5)

+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+--------------------+
|    id|                name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|               words|         CleanTokens|
+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+--------------------+
|761411|Condo in Amsterda...|4013546|   Xsj

But we don’t have a stopwordsremover for all language and contexts. Create your own list of stopwords from this text (think: what is a stopword?) Remove stopwords again, and store in column “MyCleanTokens”

In [51]:
stop_list = ['the', 'a', 'an', 'another', "for", "an", "nor", "but", "or", "yet", "so", 
                                      "in", "under", "towards", "before"]
remover = StopWordsRemover(stopWords=stop_list, inputCol='words', outputCol='MyCleanTokens')
cleanData = remover.transform(cleanData)
cleanData.show(5)

IllegalArgumentException: requirement failed: Output Column MyCleanTokens already exists.

Perform TFIDF in a new column called “VectorSpace”

In [43]:
hashingTF = HashingTF(inputCol="MyCleanTokens", outputCol="VectorSpace", numFeatures=20)
featurizedData = hashingTF.transform(cleanData)

idf = IDF(inputCol="VectorSpace", outputCol="features")
idfModel = idf.fit(featurizedData)
results = idfModel.transform(featurizedData)

results.select("MyCleanTokens", "features").show(5)

+--------------------+--------------------+
|       MyCleanTokens|            features|
+--------------------+--------------------+
|[condo, amsterdam...|(20,[1,2,7,10,11,...|
|[rental, unit, am...|(20,[1,2,5,11,12,...|
|[boat, amsterdam,...|(20,[0,1,2,5,11,1...|
|[houseboat, amste...|(20,[0,1,5,6,11,1...|
|[rental, unit, am...|(20,[1,2,9,11,12,...|
+--------------------+--------------------+


## Homework (Optional)

In a new column(‘word2vec’), repeat the procedure using word2vec instead of TF-IDF.

In [44]:
word2Vec = Word2Vec(vectorSize=20, minCount=0, inputCol="MyCleanTokens", outputCol="word2vec")
model = word2Vec.fit(results)
result = model.transform(results)

result.select("MyCleanTokens", "word2vec").show(5)

+--------------------+--------------------+
|       MyCleanTokens|            word2vec|
+--------------------+--------------------+
|[condo, amsterdam...|[0.20106191134878...|
|[rental, unit, am...|[0.38473468965717...|
|[boat, amsterdam,...|[0.19146950027117...|
|[houseboat, amste...|[0.06947926431894...|
|[rental, unit, am...|[0.30920413980881...|
+--------------------+--------------------+


Show first row word2vec vector

In [45]:
result.select("word2vec").first()

Row(word2vec=DenseVector([0.2011, 0.2439, -0.1676, -0.1466, 0.0506, -0.1991, 0.0455, 0.4709, -0.2326, 0.2024, -0.1933, -0.0018, 0.1727, -0.2837, -0.1952, -0.2497, -0.6426, -0.0711, 0.3715, -0.1627]))

## Challenge II

Take the first 500 flats in the list

In [46]:
mysample = result.limit(500)
mysample.count()

500

Find the 3 nearest neighbors for each element in that subset (candidates and query points are within the sample of 500) USING KNN

In [47]:
mysample_pd = mysample.toPandas()
tfidf = mysample_pd['features'].tolist()
text = mysample_pd['name'].tolist()
id_ = mysample_pd['id'].tolist()

# fit nearest neighbors
nbrs = NearestNeighbors(n_neighbors=4).fit(tfidf)
distances, indices = nbrs.kneighbors(tfidf[:5])

# show 3 nearest neighbors for first row except itself
print('id', [id_[i] for i in indices[0]])

id ['761411', '634170', '721291', '730916']


Find the 3 nearest neighbors for each element in that subset (candidates and query points are within the sample of 500) USING LSH with sklearn

In [48]:
# IT IS DEPRECATED or Install 3 years old version of sklearn 0.16.1
try:
    from sklearn.neighbors import LSHForest
    
    mysample_pd = mysample.toPandas()
    tfidf = mysample_pd['features'].tolist()
    text = mysample_pd['name'].tolist()
    id_ = mysample_pd['id'].tolist()
    
    lshf = LSHForest(random_state=42)
    lshf.fit(tfidf)
    
    # get the feature vectore of the first row
    query = tfidf[0]
    id_ = id_[0]
    
    # show 3 nearest neighbors for first row except itself
    distances, indices = lshf.kneighbors([query], n_neighbors=4)
    for i in range(1, len(distances[0])):
        print("distance: ", distances[0][i], "id: ", id_[indices[0][i]])
except ImportError:
    print("LSHForest could not be imported")

LSHForest could not be imported


Find the 3 nearest neighbors for each element in that subset (candidates and query points are within the sample of 500) USING LSH with pyspark

In [49]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=3)
model = mh.fit(mysample)

# get the feature vector of the first row
key =  mysample.select("features").take(1)[0].features
id_ = mysample.select("id").take(1)[0].id


# show 3 nearest neighbors for first row except itself
model.approxNearestNeighbors(mysample, key, 4).filter(col("id") != id_).show()

+-------+--------------------+-------+----------------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|     id|                name|host_id|       host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|               words|         CleanTokens|       MyCleanTokens|         VectorSpace|            features|            word2vec|              hashes|           distCol|
+-------+--------------------+-------+----------------+-------------------+-------------+--------+

## Challenge III: Homework

Repeat the LSH experiment for the full Barcelona dataset, and develop and spark boosted methodology to have an efficient parameter tuning process.

In [23]:
!wget http://data.insideairbnb.com/spain/catalonia/barcelona/2023-09-06/visualisations/listings.csv -O listings_barcelona.csv 

--2023-10-20 10:28:43--  http://data.insideairbnb.com/spain/catalonia/barcelona/2023-09-06/visualisations/listings.csv
Resolving data.insideairbnb.com (data.insideairbnb.com)... 52.217.46.139, 52.216.42.125, 52.216.106.82, ...
Connecting to data.insideairbnb.com (data.insideairbnb.com)|52.217.46.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3664972 (3.5M) [application/csv]
Saving to: ‘listings_barcelona.csv’

listings_barcelona. 100%[===================>]   3.50M  2.39MB/s    in 1.5s    

2023-10-20 10:28:45 (2.39 MB/s) - ‘listings_barcelona.csv’ saved [3664972/3664972]



In [50]:
df = spark.read.csv("listings_barcelona.csv", header=True, multiLine=True)
df.show(5)

+------+--------------------+-------+----------------+-------------------+--------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+
|    id|                name|host_id|       host_name|neighbourhood_group|       neighbourhood|         latitude|        longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|    license|
+------+--------------------+-------+----------------+-------------------+--------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+
| 18674|Rental unit in Ba...|  71615|Mireia And Maria|           Eixample|  la Sagrada Família|        

In [52]:
df.count()

18086